# MiddleWare: Dynamic Prompts

In [36]:
from typing import TypedDict

from langchain.agents import create_agent
from langchain.agents.middleware.types import modify_model_request, AgentState, ModelRequest
from langgraph.runtime import Runtime

### Using Context

In [37]:
class Context(TypedDict):
    poem_type: str

@modify_model_request
def dynamic_system_prompt(request: ModelRequest, state: AgentState, runtime: Runtime[Context]) -> ModelRequest:

    type_of_poet = runtime.context.get("poem_type", "haiku")
    request.system_prompt = "You are a helpful writer of humorous {poem_type}"

    return request

agent = create_agent(
    model="openai:gpt-4o",
    tools=[],
    middleware=[dynamic_system_prompt],
    context_schema=Context
)

In [38]:
result = agent.invoke(
    {"messages": [{"role": "user", "content": "Write me a poem about Rust"}]},
    context={"poem_type": "limerik"}
)
print(result["messages"][-1].content)

In a land where smoothness used to boast,  
There lurks a sight that haunts the coast—  
Rust, the artist with a brush so sly,  
Painting iron with its earthy dye.  

Upon old trucks and garden gates,  
It weaves its tapestry and waits.  
What once was shiny, bright, and proud,  
Is now encased in an amber shroud.  

Rust isn't hasty, oh, not too fast,  
It takes its time, ensuring the past  
Is adorned with flakes, that crunchy coat,  
Making metal crumble like a stale oat.  

It's the tattoo of time on steel,  
A mark that every car can feel.  
Wherever moisture meets with air,  
Rust follows on with savoir-faire.  

So, raise a toast to rust, my friend,  
The ultimate makeover, start to end.  
For even when things fall apart,  
Rust proves itself a work of art.  


In [39]:
result = agent.invoke(
    {"messages": [{"role": "user", "content": "Write me a poem about Rust"}]},
    context={"poem_type": "Sonnet"}
)
print(result["messages"][-1].content)

In a world where software often gets betrayed,  
There exists a language where heroes are made.  
Oh Rust, thou art valiant, a coder's delight,  
With memory safety, you banish the fright.  

C and C++ may have had their prime,  
But with their sharp edges, they wasted our time.  
Then you came along with your borrow and own,  
Preventing segfaults that made us all groan.  

With syntax so charming, you've wooed many hearts,  
Bringing order and safety, your two greatest arts.  
Concurrency's easy, and bugs fear your might,  
In the battle for flawless, you shine ever bright.  

Compile time checks that may cause some delay,  
Are truly your blessing, not painful decay.  
Each error a chance to learn and grow wise,  
While your community cheers us with muffins and pies.  

Oh Rust, how your foundation is robust!  
Through speed and stability, in you we trust.  
As we write in your verse and spread the good word,  
May your future be shiny and never deferred!


### Using State

## Setup

In [1]:
from langchain_community.utilities import SQLDatabase

db = SQLDatabase.from_uri("sqlite:///Chinook.db")

In [2]:
from langchain.chat_models import init_chat_model

llm = init_chat_model("openai:gpt-5")

In [3]:
from typing import TypedDict

# Setup your agent runtime context
class Context(TypedDict):
    first: str
    last: str
    

In [4]:
import re

from langgraph.runtime import Runtime, get_runtime
from langchain_core.tools import tool

@tool
def execute_sql(query: str) -> str:
    """Execute a SQLite command and return results.
    Named parameters like :first and :last will be filled from runtime context."""
    runtime = get_runtime(Context)
    try:
        placeholders = re.findall(r":(\w+)", query)

        if placeholders:
            params = {
                p: runtime.context[p]
                for p in placeholders
                if p in runtime.context
            }
            return db.run(query, parameters=params)
        else:
            return db.run(query)
    except Exception as e:
        return f"Error: {e}"


In [5]:
from langchain_core.tools import tool
import requests

FALLBACK_RATES = {"USD": 1.0, "EUR": 0.92, "JPY": 150.0, "GBP": 0.79}

@tool
def convert_currency(amount: float, to_currency: str) -> float:
    """Convert an amount in USD to another currency using live exchange rates.
    Always use this tool if the user requests a currency different 
    from the one stored in the database (USD)."""
    try:
        url = f"https://api.exchangerate.host/convert?from=USD&to={to_currency}&amount={amount}"
        response = requests.get(url, timeout=5)
        response.raise_for_status()
        data = response.json()
        result = data.get("result")
        if result is not None:
            return round(float(result), 2)
    except Exception as e:
        print(f"[convert_currency] Falling back due to error: {e}")

    if to_currency not in FALLBACK_RATES:
        raise ValueError("Unsupported currency in fallback mode")
    return round(amount * FALLBACK_RATES[to_currency], 2)

In [6]:
SYSTEM = """You are a careful SQLite analyst.

Rules:
- Think step-by-step.
- When you need data, call the tool `execute_sql` with ONE SELECT query.
- Read-only only; no INSERT/UPDATE/DELETE/ALTER/DROP/CREATE/REPLACE/TRUNCATE.
- Limit to 5 rows unless the user explicitly asks otherwise.
- If the tool returns 'Error:', revise the SQL and try again.
- Prefer explicit column lists; avoid SELECT *.
- If a query involves a specific customer, always use the named parameters
  :first and :last (instead of hard-coding values).
  Example: SELECT InvoiceId, Total FROM Invoice
           JOIN Customer ON Invoice.CustomerId = Customer.CustomerId
           WHERE Customer.FirstName = :first AND Customer.LastName = :last;
  Assume the customer first and last name are known and will be inserted into the query later.
"""

In [7]:
from langchain.agents import create_agent
from langchain_core.messages import SystemMessage

llm = init_chat_model("gpt-5", model_provider="openai")

agent = create_agent(
    model=llm,
    tools=[execute_sql, convert_currency],
    prompt=SystemMessage(content=SYSTEM),
    context_schema=Context,
)

## Dynamic Control of Prompts and Models

In [9]:
question = "What was the total on my last invoice?"
steps=[]

for step in agent.stream(
    {"messages": [{"role": "user", "content": question}]},
    context= {"first": "Frank", "last": "Harris"},
    stream_mode="values",
):
    step["messages"][-1].pretty_print()
    steps.append(step)

================================ Human Message =================================

What was the total on my last invoice?
================================== Ai Message ==================================
Tool Calls:
  execute_sql (call_wzJl2PAApZGAZESQxk92pJOz)
 Call ID: call_wzJl2PAApZGAZESQxk92pJOz
  Args:
    query: SELECT Invoice.InvoiceId, Invoice.InvoiceDate, Invoice.Total
FROM Invoice
JOIN Customer ON Invoice.CustomerId = Customer.CustomerId
WHERE Customer.FirstName = :first AND Customer.LastName = :last
ORDER BY Invoice.InvoiceDate DESC, Invoice.InvoiceId DESC
LIMIT 1;
================================= Tool Message =================================
Name: execute_sql

[(374, '2013-07-04 00:00:00', 5.94)]
================================== Ai Message ==================================

Your most recent invoice (Invoice 374 on 2013-07-04) totaled $5.94 USD.


In [10]:
question = "What were the titles?"
steps=[]

for step in agent.stream(
    {"messages": [{"role": "user", "content": question}]},
    context= {"first": "Frank", "last": "Harris"},
    stream_mode="values",
):
    step["messages"][-1].pretty_print()
    steps.append(step)

================================ Human Message =================================

What were the titles?
================================== Ai Message ==================================

Could you clarify which titles you mean?
- Employee job titles (from the Employee table)
- Album titles
- Track titles (song names)
- Something else (e.g., playlist names)

If it’s for a specific customer’s purchases, please provide the customer’s first and last name and any time/filter (e.g., last invoice, all time).


## Add memory

In [11]:
from langgraph.checkpoint.memory import InMemorySaver

In [12]:
from langchain.agents import create_agent
from langchain_core.messages import SystemMessage

llm = init_chat_model("gpt-5", model_provider="openai")

agent = create_agent(
    model=llm,
    tools=[execute_sql, convert_currency],
    prompt=SystemMessage(content=SYSTEM),
    context_schema=Context,
    checkpointer = InMemorySaver(),
)

In [15]:
question = "What was the total on my last invoice?"
steps=[]

for step in agent.stream(
    {"messages": [{"role": "user", "content": question}]},
    {"configurable": {"thread_id": "1"}},
    context= {"first": "Frank", "last": "Harris"},
    stream_mode="values",
):
    step["messages"][-1].pretty_print()
    steps.append(step)

================================ Human Message =================================

What was the total on my last invoice?
================================== Ai Message ==================================
Tool Calls:
  execute_sql (call_Hn8akHWS0gzz3LezPzhBZgzo)
 Call ID: call_Hn8akHWS0gzz3LezPzhBZgzo
  Args:
    query: SELECT i.InvoiceId, i.InvoiceDate, i.Total
FROM Invoice AS i
JOIN Customer AS c ON i.CustomerId = c.CustomerId
WHERE c.FirstName = :first AND c.LastName = :last
ORDER BY i.InvoiceDate DESC, i.InvoiceId DESC
LIMIT 1;
================================= Tool Message =================================
Name: execute_sql

[(374, '2013-07-04 00:00:00', 5.94)]
================================== Ai Message ==================================

$5.94


In [16]:
question = "What were the titles?"
steps=[]

for step in agent.stream(
    {"messages": [{"role": "user", "content": question}]},
    {"configurable": {"thread_id": "1"}},
    context= {"first": "Frank", "last": "Harris"},
    stream_mode="values",
):
    step["messages"][-1].pretty_print()
    steps.append(step)

================================ Human Message =================================

What were the titles?
================================== Ai Message ==================================
Tool Calls:
  execute_sql (call_C8WZtNsUGv00p7YFgsQQDtL8)
 Call ID: call_C8WZtNsUGv00p7YFgsQQDtL8
  Args:
    query: SELECT t.Name AS TrackTitle
FROM InvoiceLine AS il
JOIN Track AS t ON il.TrackId = t.TrackId
WHERE il.InvoiceId = (
  SELECT i.InvoiceId
  FROM Invoice AS i
  JOIN Customer AS c ON i.CustomerId = c.CustomerId
  WHERE c.FirstName = :first AND c.LastName = :last
  ORDER BY i.InvoiceDate DESC, i.InvoiceId DESC
  LIMIT 1
)
ORDER BY il.InvoiceLineId ASC
LIMIT 5;
================================= Tool Message =================================
Name: execute_sql

[('Holier Than Thou',), ('Through The Never',), ('My Friend Of Misery',), ('The Wait',), ('Blitzkrieg',)]
================================== Ai Message ==================================

- Holier Than Thou
- Through The Never
- My Friend